## DM lesson4

Dans ce dataset: https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv, chaque ligne correspond à un produit alimentaire mis en vente par un utilisateur.

Objectif: cleaner le dataset.

- On aimerait avoir une colonne de prix unifiés en euros. Problème: la currency n'est pas indiquée pour tous les produits: il va falloir essayer de "deviner" les currency manquantes, en se basant sur l'adresse IP de l'utilisateur.
- La colonne "infos" liste des ingrédients présents dans le produit. On préfèrerait avoir une colonne de type bool par ingrédient, indiquant si le produit contient ou non cet ingrédient.

Voici une liste d'APIs qui peut vous être utile : https://github.com/public-apis/public-apis (mais vous pouvez en utiliser d'autres si vous le voulez).

In [1]:
import pandas as pd
import requests
import numpy as np

 # 1. Converts the currency according to IP address

- Convertion approach :
ip_address -> country_code -> restcoutries_data -> currency_code -> rate -> price to euro 

In [2]:
# Load data
filename = "products.csv"
df = pd.read_csv(filename, sep=";")
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


- ip_adrress to country_code

In [3]:
# Use IPSTACK API
def find_country(IP):
    # Free API KEY from IPSTACK
    API_KEY = 'cb40520bb1a340f241911bfb00ecacb0'
    url = f'http://api.ipstack.com/{IP}?access_key={API_KEY}'
    # Request the url to get result
    data = requests.get(url).json()
    return data['country_code']

In [4]:
# regex gets a series of boolean values
ip_ok = df.ip_address.str.contains(
    '^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$')

# use boolean values above to filter ip_address with correct formats
# for wrong formats, filled with null values 
country_code = [find_country(ip) for ip in df[ip_ok].ip_address]

# add new column 'country_code'
df.loc[ip_ok, 'country_code'] = country_code

/Users/zduan/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1843: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [5]:
# check if elements in df are real, return boolean
df.applymap(np.isreal)

,username,ip_address,product,price,infos,country_code
0,False,False,False,False,False,True
1,False,False,False,False,False,True
2,False,False,False,False,False,True
3,False,False,False,False,False,True
4,False,False,False,False,False,False
...,...,...,...,...,...,...
195,False,False,False,False,False,False
196,False,False,False,False,False,True
197,False,False,False,False,False,False
198,False,False,False,False,False,False


- Convert price to euros according to country codes 

In [6]:
# get the rates of foreign currency (1 euro = rate foreign currency)
rates = requests.get('https://api.exchangeratesapi.io/latest').json()['rates']
print(rates)

{'CAD': 1.4511, 'HKD': 8.7048, 'ISK': 138.3, 'PHP': 56.951, 'DKK': 7.4704, 'HUF': 328.87, 'CZK': 25.568, 'AUD': 1.627, 'RON': 4.7553, 'SEK': 10.7445, 'IDR': 15588.67, 'INR': 78.7515, 'BRL': 4.4732, 'RUB': 70.824, 'HRK': 7.453, 'JPY': 120.59, 'THB': 33.504, 'CHF': 1.1019, 'SGD': 1.514, 'PLN': 4.2777, 'BGN': 1.9558, 'TRY': 6.4012, 'CNY': 7.8524, 'NOK': 10.1865, 'NZD': 1.7456, 'ZAR': 16.2387, 'USD': 1.1107, 'MXN': 21.202, 'ILS': 3.9242, 'GBP': 0.86598, 'KRW': 1304.87, 'MYR': 4.6494}


In [7]:
# get the country information 
restcountries_data = requests.get('https://restcountries.eu/rest/v2/all').json()
restcountries_data[0]

{'name': 'Afghanistan',
 'topLevelDomain': ['.af'],
 'alpha2Code': 'AF',
 'alpha3Code': 'AFG',
 'callingCodes': ['93'],
 'capital': 'Kabul',
 'altSpellings': ['AF', 'Afġānistān'],
 'region': 'Asia',
 'subregion': 'Southern Asia',
 'population': 27657145,
 'latlng': [33.0, 65.0],
 'demonym': 'Afghan',
 'area': 652230.0,
 'gini': 27.8,
 'timezones': ['UTC+04:30'],
 'borders': ['IRN', 'PAK', 'TKM', 'UZB', 'TJK', 'CHN'],
 'nativeName': 'افغانستان',
 'numericCode': '004',
 'currencies': [{'code': 'AFN', 'name': 'Afghan afghani', 'symbol': '؋'}],
 'languages': [{'iso639_1': 'ps',
   'iso639_2': 'pus',
   'name': 'Pashto',
   'nativeName': 'پښتو'},
  {'iso639_1': 'uz',
   'iso639_2': 'uzb',
   'name': 'Uzbek',
   'nativeName': 'Oʻzbek'},
  {'iso639_1': 'tk',
   'iso639_2': 'tuk',
   'name': 'Turkmen',
   'nativeName': 'Türkmen'}],
 'translations': {'de': 'Afghanistan',
  'es': 'Afganistán',
  'fr': 'Afghanistan',
  'ja': 'アフガニスタン',
  'it': 'Afghanistan',
  'br': 'Afeganistão',
  'pt': 'Afegan

In [8]:
# map for country_code to currency code 
countrycode_to_currency = {
    country['alpha2Code']: country['currencies'][0]['code'] for country in restcountries_data}

In [9]:
# clean and convert <price> column from string to float 
splitted = df['price'].str.split(' ')
# take the first element in splitted string list and convert it to float type 
price = splitted.str[0].astype(float)
# take the second to be the currency
currency = splitted.str[1]

df['price'] = price
df['currency'] = currency

# dict-like dataframe.replace with 'inplace = False (default)'  <=> apply filter function
df['currency_guessed'] = df['country_code'].replace(countrycode_to_currency)

In [10]:
# fill the null values of first dataframe with the second dataframe, if not null then keep the origin
df['currency'] = df['currency'].combine_first(df['currency_guessed'])

# keep only the rows where we have a crrency:
df = df[df.currency.notna()].reset_index()

In [11]:
# calculate the price in euro
df['price_euro_precise'] = df.price / df.currency.map(rates)
df['price_euro'] = df['price_euro_precise'].round(1)

In [12]:
df.head()

,index,username,ip_address,product,price,infos,country_code,currency,currency_guessed,price_euro_precise,price_euro
0,4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,JPY,7.876192,7.9
1,5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,None,MGA,None,NaN,NaN
2,7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,MXN,5.261768,5.3
3,8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,MXN,4.152910,4.2
4,10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,"Contains peanut, sugar, egg and fish",US,USD,USD,870.027910,870.0


## 2. Split the infos column to ingredients columns and fill the boolean values to check if the ingredient exist in the product.

In [13]:
df.head(20).infos

0                May contain sugar, egg and fish
1                    Ingredients: sugar and milk
2                                 Contains sugar
3              Ingredients: sugar, milk and fish
4           Contains peanut, sugar, egg and fish
5           Ingredients: mustard, sugar and milk
6                  Ingredients: gluten and sugar
7                             May contain peanut
8      Ingredients: gluten, soja, sugar and milk
9      Ingredients: gluten, soja, sugar and milk
10                   Ingredients: soja and sugar
11                Contains peanut, sugar and egg
12                       Contains sugar and milk
13    Contains gluten, soja, sugar, egg and fish
14                   Ingredients: soja and sugar
15           Ingredients: peanut, soja and sugar
16                            Ingredients: sugar
17                 Contains sugar, milk and fish
18                    May contain sugar and milk
19                     Ingredients: egg and fish
Name: infos, dtype: 

In [14]:
"""make sure only one word before the ingredients
   split by the space and the output is limited to one list(parameter 1), drop the first element
   replace 'space + and' by ','
"""
Ingredients = df.infos\
    .str.replace('May contain', 'Contains')\
    .str.split(' ', 1).str[1]\
    .str.replace(' and', ',')\
    .str.get_dummies(sep = ', ')

In [15]:
# concat the dummies matix with initial df
result = pd.concat([df, Ingredients], axis=1)

In [16]:
result.head()

,index,username,ip_address,product,price,infos,country_code,currency,currency_guessed,price_euro_precise,price_euro,egg,fish,gluten,milk,mustard,peanut,soja,sugar
0,4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",JP,JPY,JPY,7.876192,7.9,1,1,0,0,0,0,0,1
1,5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,None,MGA,None,NaN,NaN,0,0,0,1,0,0,0,1
2,7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,MX,MXN,MXN,5.261768,5.3,0,0,0,0,0,0,0,1
3,8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",MX,MXN,MXN,4.152910,4.2,0,1,0,1,0,0,0,1
4,10,cjagielaa,75.254.207.163,Cinnamon Rolls,966.34,"Contains peanut, sugar, egg and fish",US,USD,USD,870.027910,870.0,1,1,0,0,0,1,0,1
